In [ ]:
import pandas as pd
from src.plot import *
import numpy as np

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
df = pd.read_csv('loan.csv')

### EDA

#### Numerical columns

In [ ]:
numerical = df.select_dtypes('number').columns
df[numerical].isna().sum()

In [ ]:
for c in numerical:
    plot_histogram(df,variable=c)
    plt.show()

In [ ]:
df.describe()

##### Observation & to be checked: 
1. maximum **interest** rate is almost 31%, is it inline or is it too high (i.e in Poland might be too high)?  

2. **istallement** distribution looks ok at first glance,   

3. **annual income**: check outliers and missing values   

4. dti: why min value it is negative, check in detail the distribution, there are NA check why

5. recoveries: check how many of them were above 0

3. Annual income check

In [ ]:
df['annual_inc'].quantile([0.8,0.85,0.9,0.95, 0.99])

In [ ]:
plot_histogram(df[df['annual_inc']<=100000],variable='annual_inc')
# tendency in round ammounts -> risk typology 
# are high value customer risky? maybe any VIP target model?
plot_histogram(df[ (df['annual_inc']>=1000000)  & (df['annual_inc']<7000000) ],variable='annual_inc')


In [ ]:
df[df['annual_inc']>270000].sort_values('annual_inc', ascending = False).head(50)
df['annual_inc'].value_counts().tail(30)
df[df['annual_inc'].isna()]

In [ ]:
#column I need -> total loan amount, start and end date, if laast payment equals end date, convert annual to monthly, compare dti
# if paid on time or not ? 


4. dti: why min value it is negative, check in detail the distribution, there are NA check why

In [ ]:
df[df['dti']<0]


In [ ]:
plot_histogram(df[df['dti']<=50],variable='dti')
#dti <40 is in distribution
df[df['dti']>300]


In [ ]:
df[df['dti'].isna()].count()
df[(df['annual_inc']==0) & (~df['dti'].isna())]
#annual inc is zero -> dti is zero something strange


4. Recoveries

In [ ]:
df[df['recoveries']>0].count()/df.shape[0]
# Only of total is recovered, calculate how many of default are recovered

In [ ]:
plot_histogram(df[df['recoveries']>0],variable='recoveries')
# smaller ammounts are recovered

### Categorical

df.head()

In [ ]:
df.head()
dates = ['issue_d','earliest_cr_line','last_pymnt_d']

In [ ]:
categorical = df.select_dtypes('object').columns
categorical = [c for c in categorical if c not in dates]


In [ ]:
for c in categorical:
    plot_cnt(df,variable=c)
    plt.show()

In [ ]:
df['loan_status'].value_counts()
df['term'].value_counts()
df['home_ownership'].value_counts()
df['loan_status'].value_counts()